# Advanced implementation

Author: Frankie Inguanez <br />
Date: 13/01/2023<br /><br />

An advanced implementation of a 9x9 sudoku puzzle taking inspiration for 3rd parties.

In [ ]:
# Finds the next next unknown value in a sudoku 9x9 puzzle in a raster pattern.
def findEmptyRaster(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if puzzle[i][j] == 0:
                return (i, j)  # row, col
    return None

In [ ]:
# Finds the next next unknown value in a sudoku 9x9 puzzle in a spiral pattern.
def findEmptySpiral(puzzle):
    n = len(puzzle)
    x, y = 0, 0
    dx, dy = 0, 1
    for i in range(n**2):
        if puzzle[x][y] == 0:
            return (x, y)
        if x + dx < 0 or x + dx == n or y + dy < 0 or y + dy == n or puzzle[x+dx][y+dy] != 0:
            dx, dy = dy, -dx
        x += dx
        y += dy
    return None


In [ ]:
def findEmpty(puzzle, method):
    if method == 1:
        return findEmptyRaster(puzzle)
    elif method == 2:
        return findEmptySpiral(puzzle)

    raise ValueError("Unrecognized method {:0.0f} for find empty.".format(method)) 